In [1]:
import os
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from skimage.io import imread, imsave
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import random
import cv2
from skimage.util import random_noise
import numpy as np

In [7]:
mapping = pd.read_csv('mapping_images_name.csv')
classes = ['i', 'ii', 'iii', 'iv', 'ix', 'v', 'vi', 'vii', 'viii', 'x']
train_frac = [0.99, 0.95, 0.9, 0.85, 0.8, 0.75]

In [1]:
### Changes based on manual examination of the data

images_to_delete = ['i_39', 'i_97', 'i_98', 'i_165', 'i_178', 'i_227', 'ii_34', 'ii_35', 'ii_48', 'ii_43', 'ii_60', 'ii_126', 'ii_96','ii_130', 'iii_0', 'iii_3', 'iii_118', 'iii_108', 'iii_101' , 'ix_37', 'ix_61', 'ix_94', 'ix_78', 'ix_72', 'ix_116','ix_112', 'ix_111', 'ix_181', 'ix_164', 'ix_163', 'ix_184', 'ix_193', 'v_21', 'v_51', 'v_47', 'v_120', 'v_157', 'v_130','v_183', 'v_188', 'v_192', 'vi_81', 'vi_82', 'vi_109', 'vi_108', 'vi_121', 'vi_124', 'vi_156', 'vi_166', 'vii_16', 'vii_4','vii_6', 'vii_56', 'vii_54', 'vii_39', 'vii_63', 'vii_68', 'vii_99', 'vii_73', 'vii_154', 'vii_187', 'viii_26', 'viii_35','viii_37', 'viii_58', 'viii_82', 'viii_71', 'viii_149', 'viii_149', 'viii_141', 'viii_163', 'viii_161', 'viii_174', 'x_38','x_47', 'x_83', 'x_111', 'x_112', 'x_160', 'x_162', 'iv_63', 'iv_47', 'iv_94', 'iv_76', 'iv_120', 'iv_137', 'iv_152','iv_199', 'iv_241', 'iv_269', 'iv_272', 'iv_273']
images_to_move = {'i_17': 'ii', 'i_248':'v', 'i_223':'ii', 'ii_10':'v', 'ii_77':'v', 'iii_126':'ii', 'iii_182':'ii', 'ii_125':'iii', 'ix_62':'iv','ix_136':'iii', 'vi_45':'iv', 'vi_64':'iv', 'viii_31':'ii', 'viii_77':'iv', 'viii_127':'ii', 'iv_17':'x'}
images_to_move_by_folder = {'ii': ['i_17', 'i_223','iii_126', 'iii_182', 'viii_31', 'viii_127'], 'iii':['ii_125', 'ix_136'], 'iv':['ix_62', 'vi_45', 'vi_64', 'viii_77'], 
              'v':['ii_10', 'i_248', 'ii_77'], 'x':['iv_17']}

In [9]:
max_values = {'i':262, 'ii':158, 'iii':187, 'iv':282, 'ix':235,'v':197, 'vi':182, 'vii':194, 'viii':200,'x':180}

for k, v in images_to_move.items():
    images_to_delete.append(k)
    
images_to_delete = [x + '.png' for x in images_to_delete]

### data v3

##### adding images with noise. 50 images for testing.

In [26]:
#with noise

for i in range(4):
    new_folder_name = f"data_v3/noise_iter_{i}"
    os.mkdir(new_folder_name)
    os.mkdir(f"{new_folder_name}/train")
    os.mkdir(f"{new_folder_name}/val")

    for c in classes:
        old_path = f"all_data/{c}"
        new_path_val = f"{new_folder_name}/val/{c}"
        new_path_train = f"{new_folder_name}/train/{c}"

        images_c = mapping[mapping['folder'] == c]['new_name'].to_list()
        images_c = [ic for ic in images_c if ic not in images_to_delete]

        if c in images_to_move_by_folder.keys():
            counter = max_values[c]
            for img_name in images_to_move_by_folder[c]:
                wrong_path = img_name.split(sep='_')[0]
                img = imread(f"all_data/{wrong_path}/{img_name}.png")
                new_image_name = f"{c}_{counter}.png"
                counter += 1
                imsave(f"{old_path}/{new_image_name}", img)
                images_c.append(new_image_name)
                print(img_name, new_image_name)

        random.shuffle(images_c)
        x_test = images_c[:50]
        x_train = images_c[50:]

        #x_train ,x_test = train_test_split(images_c, train_size=t_f)

        os.mkdir(new_path_val)
        os.mkdir(new_path_train)

        for img_name in x_test: 
            img = imread(f"{old_path}/{img_name}")
            imsave(f"{new_path_val}/{img_name}", img)

        counter_train = len(x_train)
        for img_name in x_train:
            img_arr = imread(f"{old_path}/{img_name}")
            imsave(f"{new_path_train}/{img_name}", img_arr)
            gauss = np.random.normal(0,1,img_arr.size)
            gauss = gauss.reshape(img_arr.shape[0],img_arr.shape[1]).astype('uint8')
             # Add the Gaussian noise to the image
            img_gauss = cv2.add(img_arr,gauss) # first option for noise
            imsave(f"{new_path_train}/n_{img_name}", img_gauss)
#                 for i in range(0, 2):
#                     for j in range(0, 2):
#                         cropped_img = img_wo_border[j:j+(height-2), i:i+(width-2)]
#                         imsave(f"{new_path_train}/{i}_{j}_{img_name}", cropped_img)
#                         counter_train += 1
#                         if counter_train == 950:
#                             break
#                     if counter_train == 950:
#                         break
#                 if counter_train == 950:
#                     break
            
#             for img_name in x_train: 
#                 img = imread(f"{old_path}/{img_name}")
#                 imsave(f"{new_path_train}/{img_name}", img)

/tmp/ipykernel_12653/1859473456.py:39: UserWarning: data_v3/noise_iter_0/val/i/i_127.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img)
/tmp/ipykernel_12653/1859473456.py:39: UserWarning: data_v3/noise_iter_0/val/i/i_254.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img)
/tmp/ipykernel_12653/1859473456.py:39: UserWarning: data_v3/noise_iter_0/val/i/i_174.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img)
/tmp/ipykernel_12653/1859473456.py:44: UserWarning: data_v3/noise_iter_0/train/i/i_63.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_12653/1859473456.py:49: UserWarning: data_v3/noise_iter_0/train/i/n_i_63.png is a low contrast image
  imsave(f"{new_path_train}/n_{img_name}", img_gauss)
/tmp/ipykernel_12653/1859473456.py:44: UserWarning: data_v3/noise_iter_0/train/i/i_210.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_12653/185947345

i_17 ii_158.png
i_223 ii_159.png
iii_126 ii_160.png
iii_182 ii_161.png
viii_31 ii_162.png
viii_127 ii_163.png
ii_125 iii_187.png
ix_136 iii_188.png


/tmp/ipykernel_12653/1859473456.py:39: UserWarning: data_v3/noise_iter_0/val/iii/iii_135.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img)
/tmp/ipykernel_12653/1859473456.py:44: UserWarning: data_v3/noise_iter_0/train/iii/iii_43.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_12653/1859473456.py:49: UserWarning: data_v3/noise_iter_0/train/iii/n_iii_43.png is a low contrast image
  imsave(f"{new_path_train}/n_{img_name}", img_gauss)
/tmp/ipykernel_12653/1859473456.py:44: UserWarning: data_v3/noise_iter_0/train/iii/iii_122.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_12653/1859473456.py:49: UserWarning: data_v3/noise_iter_0/train/iii/n_iii_122.png is a low contrast image
  imsave(f"{new_path_train}/n_{img_name}", img_gauss)
/tmp/ipykernel_12653/1859473456.py:44: UserWarning: data_v3/noise_iter_0/train/iii/iii_177.png is a low contrast image
  imsave(f"{new_path_train}/{img_n

ix_62 iv_282.png
vi_45 iv_283.png
vi_64 iv_284.png
viii_77 iv_285.png


/tmp/ipykernel_12653/1859473456.py:44: UserWarning: data_v3/noise_iter_0/train/ix/ix_155.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_12653/1859473456.py:49: UserWarning: data_v3/noise_iter_0/train/ix/n_ix_155.png is a low contrast image
  imsave(f"{new_path_train}/n_{img_name}", img_gauss)


...

i_17 ii_158.png
i_223 ii_159.png
iii_126 ii_160.png
iii_182 ii_161.png
viii_31 ii_162.png
viii_127 ii_163.png
ii_125 iii_187.png
ix_136 iii_188.png


/tmp/ipykernel_12653/1859473456.py:39: UserWarning: data_v3/noise_iter_3/val/iii/iii_122.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img)
/tmp/ipykernel_12653/1859473456.py:39: UserWarning: data_v3/noise_iter_3/val/iii/iii_43.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img)
/tmp/ipykernel_12653/1859473456.py:44: UserWarning: data_v3/noise_iter_3/train/iii/iii_177.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_12653/1859473456.py:49: UserWarning: data_v3/noise_iter_3/train/iii/n_iii_177.png is a low contrast image
  imsave(f"{new_path_train}/n_{img_name}", img_gauss)
/tmp/ipykernel_12653/1859473456.py:44: UserWarning: data_v3/noise_iter_3/train/iii/iii_135.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_12653/1859473456.py:49: UserWarning: data_v3/noise_iter_3/train/iii/n_iii_135.png is a low contrast image
  imsave(f"{new_path_train}/n_{img_name}", im

ix_62 iv_282.png
vi_45 iv_283.png
vi_64 iv_284.png
viii_77 iv_285.png


/tmp/ipykernel_12653/1859473456.py:44: UserWarning: data_v3/noise_iter_3/train/ix/ix_155.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_12653/1859473456.py:49: UserWarning: data_v3/noise_iter_3/train/ix/n_ix_155.png is a low contrast image
  imsave(f"{new_path_train}/n_{img_name}", img_gauss)


ii_10 v_197.png
i_248 v_198.png
ii_77 v_199.png
iv_17 x_180.png


### data v4

##### adding images with noise + adding extra images to the train (cropping)

In [11]:
#with noise and cuts (v4)
data_v4_folders = []
for test_size in [50, 60, 70]:
    for i in range(3):
        new_folder_name = f"data_v4/test_size_{test_size}_iter_{i}"
        data_v4_folders.append(new_folder_name)
        os.mkdir(new_folder_name)
        os.mkdir(f"{new_folder_name}/train")
        os.mkdir(f"{new_folder_name}/val")

        for c in classes:
            old_path = f"all_data/{c}"
            new_path_val = f"{new_folder_name}/val/{c}"
            new_path_train = f"{new_folder_name}/train/{c}"

            images_c = mapping[mapping['folder'] == c]['new_name'].to_list()
            images_c = [ic for ic in images_c if ic not in images_to_delete]

            if c in images_to_move_by_folder.keys():
                counter = max_values[c]
                for img_name in images_to_move_by_folder[c]:
                    wrong_path = img_name.split(sep='_')[0]
                    img = imread(f"all_data/{wrong_path}/{img_name}.png")
                    new_image_name = f"{c}_{counter}.png"
                    counter += 1
                    imsave(f"{old_path}/{new_image_name}", img)
                    images_c.append(new_image_name)
                    print(img_name, new_image_name)

            random.shuffle(images_c)
            x_test = images_c[:test_size]
            x_train = images_c[test_size:]

            #x_train ,x_test = train_test_split(images_c, train_size=t_f)

            os.mkdir(new_path_val)
            os.mkdir(new_path_train)

            for img_name in x_test: 
                img = imread(f"{old_path}/{img_name}")
                imsave(f"{new_path_val}/{img_name}", img)
            

            counter_train = len(x_train)
            
            #saving again the original image and the image with noise
            for img_name in x_train:
                img_arr = imread(f"{old_path}/{img_name}")
                imsave(f"{new_path_train}/{img_name}", img_arr)
                gauss = np.random.normal(0,1,img_arr.size)
                gauss = gauss.reshape(img_arr.shape[0],img_arr.shape[1]).astype('uint8')
                 # Add the Gaussian noise to the image
                img_gauss = cv2.add(img_arr,gauss) # first option for noise
                imsave(f"{new_path_train}/n_{img_name}", img_gauss)
                counter_train += 1
            
            # adding cuts of the original image
            for img_name in x_train:
                img_arr = imread(f"{new_path_train}/{img_name}")
                height, width = img_arr.shape[0], img_arr.shape[1]
                border_w = width // 10
                border_h = height // 10
                img_wo_border = img_arr[border_h:height-border_h, border_w:width-border_w]
                height, width = img_wo_border.shape[0], img_wo_border.shape[1]
                for i in [2,4]:
                    for j in [2,4]:
                        cropped_img = img_wo_border[j:j+(height-2), i:i+(width-2)]
                        imsave(f"{new_path_train}/{i}_{j}_{img_name}", cropped_img)
                        counter_train += 1
                        if counter_train == 1000-test_size:
                            break
                    if counter_train == 1000-test_size:
                        break
                if counter_train == 1000-test_size:
                    break

                

/tmp/ipykernel_3577/405195164.py:41: UserWarning: data_v4/test_size_50_iter_0/val/i/i_57.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img)
/tmp/ipykernel_3577/405195164.py:41: UserWarning: data_v4/test_size_50_iter_0/val/i/i_250.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img)
/tmp/ipykernel_3577/405195164.py:54: UserWarning: data_v4/test_size_50_iter_0/train/i/n_i_93.png is a low contrast image
  imsave(f"{new_path_train}/n_{img_name}", img_gauss)
/tmp/ipykernel_3577/405195164.py:49: UserWarning: data_v4/test_size_50_iter_0/train/i/i_180.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_3577/405195164.py:54: UserWarning: data_v4/test_size_50_iter_0/train/i/n_i_180.png is a low contrast image
  imsave(f"{new_path_train}/n_{img_name}", img_gauss)
/tmp/ipykernel_3577/405195164.py:54: UserWarning: data_v4/test_size_50_iter_0/train/i/n_i_217.png is a low contrast image
  imsave(f"{new_path_train}/n_{img

i_17 ii_158.png
i_223 ii_159.png
iii_126 ii_160.png
iii_182 ii_161.png
viii_31 ii_162.png
viii_127 ii_163.png
ii_125 iii_187.png
ix_136 iii_188.png


/tmp/ipykernel_3577/405195164.py:41: UserWarning: data_v4/test_size_50_iter_0/val/iii/iii_177.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img)
/tmp/ipykernel_3577/405195164.py:41: UserWarning: data_v4/test_size_50_iter_0/val/iii/iii_43.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img)
/tmp/ipykernel_3577/405195164.py:49: UserWarning: data_v4/test_size_50_iter_0/train/iii/iii_122.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_3577/405195164.py:54: UserWarning: data_v4/test_size_50_iter_0/train/iii/n_iii_122.png is a low contrast image
  imsave(f"{new_path_train}/n_{img_name}", img_gauss)
/tmp/ipykernel_3577/405195164.py:54: UserWarning: data_v4/test_size_50_iter_0/train/iii/n_iii_50.png is a low contrast image
  imsave(f"{new_path_train}/n_{img_name}", img_gauss)
/tmp/ipykernel_3577/405195164.py:49: UserWarning: data_v4/test_size_50_iter_0/train/iii/iii_135.png is a low contrast image
  imsave(f"{

ix_62 iv_282.png
vi_45 iv_283.png
vi_64 iv_284.png
viii_77 iv_285.png


/tmp/ipykernel_3577/405195164.py:49: UserWarning: data_v4/test_size_50_iter_0/train/ix/ix_155.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_3577/405195164.py:54: UserWarning: data_v4/test_size_50_iter_0/train/ix/n_ix_155.png is a low contrast image
  imsave(f"{new_path_train}/n_{img_name}", img_gauss)


...

i_17 ii_158.png
i_223 ii_159.png
iii_126 ii_160.png
iii_182 ii_161.png
viii_31 ii_162.png
viii_127 ii_163.png
ii_125 iii_187.png
ix_136 iii_188.png


/tmp/ipykernel_3577/405195164.py:41: UserWarning: data_v4/test_size_70_iter_2/val/iii/iii_122.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img)
/tmp/ipykernel_3577/405195164.py:41: UserWarning: data_v4/test_size_70_iter_2/val/iii/iii_135.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img)
/tmp/ipykernel_3577/405195164.py:49: UserWarning: data_v4/test_size_70_iter_2/train/iii/iii_177.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_3577/405195164.py:54: UserWarning: data_v4/test_size_70_iter_2/train/iii/n_iii_177.png is a low contrast image
  imsave(f"{new_path_train}/n_{img_name}", img_gauss)
/tmp/ipykernel_3577/405195164.py:49: UserWarning: data_v4/test_size_70_iter_2/train/iii/iii_43.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_3577/405195164.py:54: UserWarning: data_v4/test_size_70_iter_2/train/iii/n_iii_43.png is a low contrast image
  imsave(f"{new_

ix_62 iv_282.png
vi_45 iv_283.png
vi_64 iv_284.png
viii_77 iv_285.png


/tmp/ipykernel_3577/405195164.py:49: UserWarning: data_v4/test_size_70_iter_2/train/ix/ix_155.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_3577/405195164.py:54: UserWarning: data_v4/test_size_70_iter_2/train/ix/n_ix_155.png is a low contrast image
  imsave(f"{new_path_train}/n_{img_name}", img_gauss)


ii_10 v_197.png
i_248 v_198.png
ii_77 v_199.png
iv_17 x_180.png


In [23]:
# for i in range(4):
#         new_folder_name = f"v3/iter_{i}"
#         os.mkdir(new_folder_name)
#         os.mkdir(f"{new_folder_name}/train")
#         os.mkdir(f"{new_folder_name}/val")

#         for c in classes:
#             old_path = f"all_data/{c}"
#             new_path_val = f"{new_folder_name}/val/{c}"
#             new_path_train = f"{new_folder_name}/train/{c}"
            
#             images_c = mapping[mapping['folder'] == c]['new_name'].to_list()
#             images_c = [ic for ic in images_c if ic not in images_to_delete]
            
#             if images_to_move_by_folder[c]:
#                 counter = max_values[c]
#                 for img_name in images_to_move_by_folder[c]:
#                     wrong_path = img_name.split(sep='_')[0]
#                     img = imread(f"{wrong_path}/{img_name}")
#                     new_image_name = {f"{c}_{counter}.png"}
#                     counter += 1
#                     imsave(f"{old_path}/{new_image_name}", img)
#                     images_c.append(new_image_name)
#                     print(img_name, new_image_name)
                    
#             random.shuffle(images_c)
#             x_test = images_c[:50]
#             x_train = images_c[50:]
            
#             #x_train ,x_test = train_test_split(images_c, train_size=t_f)
            
#             os.mkdir(new_path_val)
#             os.mkdir(new_path_train)
            
#             for img_name in x_test: 
#                 img = imread(f"{old_path}/{img_name}")
#                 imsave(f"{new_path_val}/{img_name}", img)
                
#             counter_train = len(x_train)
#             for img_name in x_train:
#                 img_arr = imread(f"{new_path_train}/{img_name}")
#                 height, width = img_arr.shape[0], img_arr.shape[1]
#                 border_w = width // 10
#                 border_h = height // 10
#                 img_wo_border = img_arr[border_h:height-border_h, border_w:width-border_w]
#                 height, width = img_wo_border.shape[0], img_wo_border.shape[1]
#                 for i in range(0, 2):
#                     for j in range(0, 2):
#                         cropped_img = img_wo_border[j:j+(height-2), i:i+(width-2)]
#                         imsave(f"{new_path_train}/{i}_{j}_{img_name}", cropped_img)
#                         counter_train += 1
#                         if counter_train == 950:
#                             break
#                     if counter_train == 950:
#                         break
#                 if counter_train == 950:
#                     break
            
# #             for img_name in x_train: 
# #                 img = imread(f"{old_path}/{img_name}")
# #                 imsave(f"{new_path_train}/{img_name}", img)

/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.99_iter_0/train/i/i_254.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.99_iter_0/train/i/i_127.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.99_iter_0/train/i/i_57.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.99_iter_0/train/i/i_180.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.99_iter_0/train/i/i_210.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.99_iter_0/train/i/i_110.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/46480448

/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.99_iter_2/train/iii/iii_135.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.99_iter_2/train/ix/ix_155.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.99_iter_2/train/vi/vi_156.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:21: UserWarning: train_frec_0.95_iter_0/val/i/i_127.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:21: UserWarning: train_frec_0.95_iter_0/val/i/i_174.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.95_iter_0/train/i/i_57.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/46480448

/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.95_iter_2/train/iii/iii_177.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.95_iter_2/train/iii/iii_135.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.95_iter_2/train/iii/iii_43.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.95_iter_2/train/iii/iii_122.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.95_iter_2/train/ix/ix_155.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.95_iter_2/train/vi/vi_156.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipyk

/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.9_iter_2/train/i/i_127.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.9_iter_2/train/i/i_79.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.9_iter_2/train/iii/iii_122.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.9_iter_2/train/iii/iii_135.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.9_iter_2/train/iii/iii_177.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.9_iter_2/train/iii/iii_43.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091

/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.85_iter_2/train/i/i_210.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:21: UserWarning: train_frec_0.85_iter_2/val/iii/iii_43.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:21: UserWarning: train_frec_0.85_iter_2/val/iii/iii_135.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.85_iter_2/train/iii/iii_122.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.85_iter_2/train/iii/iii_177.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.85_iter_2/train/ix/ix_155.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_1309

/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.8_iter_2/train/iii/iii_122.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.8_iter_2/train/iii/iii_43.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.8_iter_2/train/iii/iii_135.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.8_iter_2/train/iii/iii_177.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.8_iter_2/train/ix/ix_155.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:21: UserWarning: train_frec_0.8_iter_2/val/vi/vi_156.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img)
/tmp/ipykernel_1309

/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.75_iter_2/train/i/i_106.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:21: UserWarning: train_frec_0.75_iter_2/val/iii/iii_135.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:21: UserWarning: train_frec_0.75_iter_2/val/iii/iii_177.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.75_iter_2/train/iii/iii_43.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.75_iter_2/train/iii/iii_122.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_13091/464804480.py:25: UserWarning: train_frec_0.75_iter_2/train/ix/ix_155.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img)
/tmp/ipykernel_1309

In [30]:
# for t_f in train_frac:
#     for i in range(3):
#         new_folder_name = f"train_frec_{t_f}_iter_{i}_extend"
#         os.mkdir(new_folder_name)
#         os.mkdir(f"{new_folder_name}/train")
#         os.mkdir(f"{new_folder_name}/val")

#         for c in classes:
#             images_c = mapping[mapping['folder'] == c]['new_name'].to_list()
#             x_train ,x_test = train_test_split(images_c, train_size=t_f)
            
#             old_path = f"all_data/{c}"
#             new_path_val = f"{new_folder_name}/val/{c}"
#             new_path_train = f"{new_folder_name}/train/{c}"
            
#             os.mkdir(new_path_val)
#             os.mkdir(new_path_train)
            
#             for img_name in x_test: 
#                 img_arr = imread(f"{old_path}/{img_name}")
#                 imsave(f"{new_path_val}/{img_name}", img_arr)
                
#                 height, width = img_arr.shape[0], img_arr.shape[1]
#                 border_w = width // 10
#                 border_h = height // 10
#                 img_wo_border = img_arr[border_h:height-border_h, border_w:width-border_w]
#                 height, width = img_wo_border.shape[0], img_wo_border.shape[1]
#                 for i in range(0, 2):
#                     for j in range(0, 2):
#                         cropped_img = img_wo_border[j:j+(height-2), i:i+(width-2)]
#                         plt.imsave(f"{new_path_val}/{i}_{j}_{img_name}", cropped_img, cmap=cm.gray)
            
#             for img_name in x_train: 
#                 img_arr = imread(f"{old_path}/{img_name}")
#                 imsave(f"{new_path_train}/{img_name}", img_arr)
                
#                 height, width = img_arr.shape[0], img_arr.shape[1]
#                 border_w = width // 10
#                 border_h = height // 10
#                 img_wo_border = img_arr[border_h:height-border_h, border_w:width-border_w]
#                 height, width = img_wo_border.shape[0], img_wo_border.shape[1]
#                 for i in range(0, 2):
#                     for j in range(0, 2):
#                         cropped_img = img_wo_border[j:j+(height-2), i:i+(width-2)]
#                         plt.imsave(f"{new_path_train}/{i}_{j}_{img_name}", cropped_img, cmap=cm.gray)

/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.99_iter_0_extend/train/i/i_245.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.99_iter_0_extend/train/i/i_79.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.99_iter_0_extend/train/i/i_250.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.99_iter_0_extend/train/i/i_180.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.99_iter_0_extend/train/i/i_254.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.99_iter_0_extend/train/i/i_110.png is a low contrast image
  imsave(f"{n

/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.99_iter_2_extend/train/iii/iii_177.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.99_iter_2_extend/train/iii/iii_135.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.99_iter_2_extend/train/iii/iii_43.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.99_iter_2_extend/train/iii/iii_122.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.99_iter_2_extend/train/ix/ix_155.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.99_iter_2_extend/train/vi/vi_156.png is a low contrast

/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.95_iter_2_extend/train/i/i_79.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.95_iter_2_extend/train/i/i_210.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.95_iter_2_extend/train/i/i_110.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.95_iter_2_extend/train/i/i_254.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.95_iter_2_extend/train/iii/iii_122.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.95_iter_2_extend/train/iii/iii_135.png is a low contrast image
  ims

/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.9_iter_2_extend/train/i/i_180.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.9_iter_2_extend/train/i/i_250.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.9_iter_2_extend/train/i/i_63.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.9_iter_2_extend/train/i/i_210.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.9_iter_2_extend/train/i/i_79.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.9_iter_2_extend/train/i/i_106.png is a low contrast image
  imsave(f"{new_path

/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.85_iter_1_extend/train/iii/iii_177.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.85_iter_1_extend/train/iii/iii_122.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.85_iter_1_extend/train/ix/ix_155.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.85_iter_1_extend/train/vi/vi_156.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:21: UserWarning: train_frec_0.85_iter_2_extend/val/i/i_245.png is a low contrast image
  imsave(f"{new_path_val}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:21: UserWarning: train_frec_0.85_iter_2_extend/val/i/i_254.png is a low contrast image
  imsa

/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.8_iter_1_extend/train/i/i_106.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.8_iter_1_extend/train/i/i_127.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.8_iter_1_extend/train/i/i_245.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.8_iter_1_extend/train/iii/iii_122.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.8_iter_1_extend/train/iii/iii_135.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.8_iter_1_extend/train/iii/iii_43.png is a low contrast image
  imsav

/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.75_iter_1_extend/train/i/i_110.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.75_iter_1_extend/train/i/i_106.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.75_iter_1_extend/train/i/i_57.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.75_iter_1_extend/train/i/i_174.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.75_iter_1_extend/train/i/i_63.png is a low contrast image
  imsave(f"{new_path_train}/{img_name}", img_arr)
/tmp/ipykernel_13091/405615870.py:35: UserWarning: train_frec_0.75_iter_1_extend/train/i/i_180.png is a low contrast image
  imsave(f"{ne